<a href="https://colab.research.google.com/github/Tanushree-Kadgi/Deepfake-Detection-System/blob/main/deepfake_vs_real_face_detection_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Create Kaggle API to Upload the Dataset
from google.colab import files
uploaded = files.upload()


Saving kaggle.json to kaggle (2).json


In [ ]:
!pip install -q kaggle


In [ ]:
#Creates a hidden folder named .kaggle in home directory.
#Kaggle API requires a folder ~/.kaggle to store your API key.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
! pwd

/content


In [ ]:
ls

 deepfake-and-real-images.zip  'kaggle (1).json'   sample_data/
 deepfake_dataset/             'kaggle (2).json'
 drive/                         kaggle.json


In [ ]:
!kaggle datasets download -d manjilkarki/deepfake-and-real-images


Dataset URL: https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images
License(s): unknown
deepfake-and-real-images.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip deepfake-and-real-images.zip -d /content/deepfake_dataset


Archive:  deepfake-and-real-images.zip
replace /content/deepfake_dataset/Dataset/Test/Fake/fake_0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#Importing necessary libraries and modules
import warnings #for handling warnings
warnings.filterwarnings("ignore") #Ignore warnings during execution

import gc
import numpy as np
import pandas as pd
import itertools
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    f1_score
)

#Import custom modules and classes
from imblearn.over_sampling import RandomOverSampler
import accelerate
!pip install evaluate # Install the missing 'evaluate' library
import evaluate
from datasets import Dataset, Image, ClassLabel
from transformers import ( #Import various modules from the transformer library
    TrainingArguments,#For training arguments
    Trainer, #For model Training
    ViTImageProcessor, #For processing image data with ViT models
    ViTForImageClassification, #ViT model foe image classification
    DefaultDataCollator #For collating data in the default way
)

import torch #Import PyTorch for Deep Learning
from torch.utils.data import DataLoader #For creating data loaders
from torchvision.transforms import ( #Import image transformer functions
    CenterCrop, #Center crop image
    Compose, #Compose multiple image transformation
    Normalize, #Normalize image pixel values
    RandomRotation, #Apply random rotation to images
    RandomResizedCrop, #Crop and resize images randomly
    RandomHorizontalFlip, #Apply random horizontal flip
    RandomAdjustSharpness, #Adjust sharpness randomly
    Resize, #Resize images
    ToTensor #Convert images to Pytorch tensors
)

In [ ]:
#Import the necessary module from the Python Imaging Library (PIL)
from PIL import ImageFile

#Enable the option to load truncated images
#This setting allows the PIL library to attempt loding images even if they are corrupted or incomplete.
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# import necessary libraries
image_dict ={}

# Define list of file names
from pathlib import Path
from tqdm import tqdm
import os
#Initialize empty lists to store file names and labels
file_names = []
labels = []

In [ ]:

#for file in sorted((Path('D:/DEEPFAKE Detection System/Dataset/Test').glob('**/*.*'))):
    #label = str(file).split('/')[-2]

# IMPORTANT: Update this path to your actual dataset location in Google Drive
data_path = Path(r"deepfake_dataset/Dataset/Test") # Example path, adjust as needed
file_names = []
labels = []
for file in sorted(data_path.glob("**/*.*")):
    if not file.is_file():
        continue
    # rel is path relative to the Test folder, e.g. "Fake/image1.jpg" -> parts = ('Fake', 'image1.jpg')
    try:
        rel = file.relative_to(data_path)
    except Exception as e:
        # fallback: use parent name
        rel = file
    parts = rel.parts
    if len(parts) >= 1:
        label = parts[0]               # first folder under Test -> Fake or Real
    else:
        label = file.parent.name       # fallback

    file_names.append(str(file))
    labels.append(label)

print("Total files:", len(file_names), "Total labels:", len(labels))

df = pd.DataFrame({"image": file_names, "label": labels})
print(df.shape)
print(df["label"].value_counts())   # quick sanity check of labels


In [ ]:
# show first 10 rows for verification
df.head(100000)
df[['image', 'label']].head(100000)


In [ ]:
df['label'].unique()

In [ ]:
df.shape, df.columns


In [ ]:

# y contain the target variable (label) we want to predict
y = df['label']

# Drop the 'label' column from DataFrame to seperate features from target variables
df = df.drop(['label'], axis=1)

# Create a RandomOverSampler object with a specified random seed (random_state=83)
ros = RandomOverSampler(random_state=83)

#Use the RandomOverSampler to resample the dataset by oversampling the minority class
df,y_resampled = ros.fit_resample(df, y)

#Delete the original 'y' variable
del y

#Add resampled target variable
df['label'] = y_resampled

# Delete the y resampled variable as a new 'label' column is in DataFrame 'df'
del y_resampled

# Perform GC
gc.collect()

print(df.shape)

In [ ]:
# As UI is not created..This is only for TESTING purpose

'''from google.colab import files
from PIL import Image
import io
import torch
import torch.nn.functional as F

# Move model to GPU
model.to("cuda")
model.eval()

uploaded = files.upload()

for filename, data in uploaded.items():
    img = Image.open(io.BytesIO(data)).convert("RGB")
    display(img)

    # Preprocess
    inputs = processor(images=img, return_tensors="pt")
    inputs = {k: v.to("cuda") for k, v in inputs.items()}

    # Predict
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probs = F.softmax(logits, dim=-1)        # convert to probabilities
    preds = torch.argmax(probs, dim=-1).item()

    # Confidence score
    confidence = probs[0][preds].item()

    # Label
    label = model.config.id2label[preds]

    print("---------------------------------")
    print("Prediction:", label)
    print("Confidence:", round(confidence * 100, 2), "%")
    print("---------------------------------") '''


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# After running the above cell and following the instructions, you can list the contents
# of your Google Drive to verify it's mounted correctly.
# For example, to list the contents of your root Google Drive folder:
!ls /content/gdrive/MyDrive

Once your Drive is mounted, you'll need to update the `data_path` in your code to point to the correct location of your dataset within your Google Drive. For example, if your 'Test' folder is directly inside 'MyDrive', the path would be `/content/gdrive/MyDrive/DEEPFAKE Detection System/Dataset/Test`.

In [ ]:
dataset = Dataset.from_pandas(df).cast_column("image", Image())

In [ ]:
dataset[0]["image"]

In [ ]:
# Extracting subset of elements from the 'label' list using slicing
labels_subset = labels[:5]

# Printing subset of elements
print(labels_subset)

In [ ]:
#Creating list of unique lables by converting 'labels' to a set and then back to list
labels_list = ['Real', 'Fake']

#Initialize empty dictionaries to map labels to IDs and vice versa
label2id, id2label = dict(), dict()

#Iterate over the unique labels and assign each label an ID , and vice versa
for i, label in enumerate(labels_list):
    label2id[label] = i #Map label to its corresponding ID
    id2label[i] = label #Map the ID to its corresponding label

# Print the resulting dictionaries for references
print("Mapping of IDs to Labels:", id2label,'\n')
print("Mapping of Labels to IDs:", label2id)




In [ ]:
# Creating classlabels to match labels to IDs
ClassLabels = ClassLabel(num_classes=len(labels_list), names=labels_list)

#Mapping labels to IDs
def map_label2id(example):
    example['label'] = ClassLabels.str2int(example['label'])
    return example

dataset = dataset.map(map_label2id, batched=True)

#Casting label column to ClassLabel Object
dataset = dataset.cast_column('label', ClassLabels)

#Splliting the datasets into training and testing sets using 60-40 split ratio
dataset = dataset.train_test_split(test_size=0.4, shuffle=True, stratify_by_column="label")

# Extracting the training data from the split dataset
train_data = dataset['train']

# Extracting the testing data from the split dataset
test_data = dataset['test']




In [ ]:
# Define pre-trained ViT model string
model_str = "dima806/deepfake_vs_real_image_detection"

# Create processor for ViT model i/p from pretrained model
processor = ViTImageProcessor.from_pretrained(model_str)

# Rerive the image mean and standard deviation used for normalizarion
image_mean, image_std = processor.image_mean, processor.image_std

# Get size (height) of the ViT models i/p images
size = processor.size["height"]
print("Size: ",size)

# Define a normalization Transformation
normalize = Normalize(mean=image_mean, std=image_std)

#Define a Transformation for training data
_train_transforms = Compose(
    [
        Resize((size, size)),
        RandomRotation(90),
        RandomAdjustSharpness(2),
        ToTensor(),
        normalize
    ]
)

#Define a set of Transformation for validation data
_val_transforms = Compose(
    [
        Resize((size,size)),
        ToTensor(),
        normalize
    ]
)

#Define a function to apply training transformation to a batch examples
def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['image']]
    return examples


#Define a function to apply validation transformation to a batch examples
def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['image']]
    return examples


In [ ]:
# Set transformers for training datasets
train_data.set_transform(train_transforms)

# Set transformers for test datasets
test_data.set_transform(val_transforms)


In [ ]:
# Define a collate function that prepares batched data for model training
def collate_fn(examples):
    # Stack the pixel values from individual examples into a single tensor.
    pixel_values = torch.stack([example ["pixel_values"] for example in examples])

    # Convert the label strings in examples to corresponding numeric IDs using 1
    labels = torch.tensor ([example ['label'] for example in examples])

    # Return a dictionary containing the batched pixel values and labels.
    return {"pixel_values": pixel_values, "labels": labels}


In [ ]:
# Load , train and evaluate model

#Create a VITForImageClassification model from a pretrained checkpoint with a specified number of output labels.
model = ViTForImageClassification.from_pretrained (model_str, num_labels=len(labels_list))

#Configure the mapping of class labels to their corresponding indices for later reference.
model.config.id2label = id2label
model.config.label2id = label2id

#Calculate and print the number of trainable parameters in millions for the model.
print(model.num_parameters(only_trainable=True) / 1e6)

In [ ]:
#Load accuracy metric
#accuracy = evaluate.load("accuray")
from evaluate import load
accuracy = load("glue", "mrpc")

#Define a function to calculate evaluation metric
def compute_metrics(eval_pred):
    # Extract model predictions from the evaluation prediction object
    predictions = eval_pred.predictions

    # Extract true labels from the evaluation prediction object
    label_ids = eval_pred.label_ids

    # Calculate accuracy using the loaded accuracy metric
    # Convert model predictions to class labels by selecting the class with the highest probability (argmax)
    predicted_labels = predictions.argmax(axis=1)

    # Calculate accuracy score by comparing predicted labels to true labels
    acc_score = accuracy.compute(predictions=predicted_labels, references=label_ids) ['accuracy']

    # Return the computed accuracy as a dictionary with the key "accuracy"
    return {
        "accuracy": acc_score
    }


In [ ]:
!pip install --upgrade transformers


In [ ]:
from transformers import TrainingArguments

metric_name = "accuracy"
model_name ="deepfake_vs_real_image_detection"
num_train_epochs = 2

args = TrainingArguments(

    output_dir=model_name,
    logging_dir='./logs',
    eval_strategy="epoch",
    learning_rate=1e-6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=8,
    num_train_epochs=num_train_epochs,
    weight_decay=0.02,
    warmup_steps=50,
    remove_unused_columns=False,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=1,
    report_to="none"
)


In [ ]:
import transformers
from transformers import TrainingArguments

print("Transformers version:", transformers.__version__)
print("TrainingArguments is from:", TrainingArguments.__module__)


In [ ]:
from transformers import Trainer

# Create a Trainer instaance for fine-tuning a language model
trainer = Trainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor
)

In [ ]:
# Start training the model using the trainer object
trainer.train()

In [ ]:
# Evaluate the post training model
trainer.evaluate()

In [ ]:
# Use the trained 'trainer' to make predictions
outputs = trainer.predict(test_data)

# Print the metrics obtained from the predictions o/p
print(outputs.metrics)



In [ ]:
outputs = trainer.predict(test_data)

# Extract the true lablels from the model o/p
y_true = outputs.label_ids

# Predict the labels
y_pred = outputs.predictions.argmax(1)

import numpy as np
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

# Define a func to plot a confusion metrics
def plot_confusion_matrix(cm, classes,title='Confusion matrix', cmap=plt.cm.Blues, figsize=(10,8)):

    # Create a figure with a specified size
    plt.figure(figsize=figsize)

    # Display confusion metrics
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    # Define tickmarks and labels for the classes
    ticl_marks = np.arange(len(classes))
    plt.xticks(ticl_marks, classes, rotation=90)
    plt.yticks(ticl_marks, classes)

    fmt = '.0f'

    thresh = cm.max() / 2.0

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

# calculate accuracy and F1 score
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='macro')

# Display
print(f"Accuracy: {accuracy:4f}")
print(f"F1 Score: {f1:4f}")

# Get the confusion metrix if theres a small no. of labels
if(len(labels_list) <= 150):
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, labels_list, figsize=(8, 6))

# Finally display classification report
print()
print("Classification Report")
print()
print(classification_report(y_true, y_pred, target_names=labels_list, digits=4))



In [ ]:
# Save the trained model
trainer.save_model()